## DataLoader Class
The DataLoader Class loads data into a SQLite database from files. All files in the folder should be in the same format and they should be csv files or zipped csv files or any other file type in csv format

In [1]:
# Import libraries
import pandas as pd
import os
from sqlalchemy import create_engine
from loaders import DataLoader

## Parameters that could be changed

In [2]:
DIRECTORY = 'data' # Folder in which files are saved
DBNAME = 'data.sqlite3' # Name of the database
TABLENAME = 'eod' # Name of the table
UNIVERSE = 'NIFTY50'
STOP_LOSS = 4
NUM_STOCKS = 5
CAPITAL = 20000
LEVERAGE = 1
REF_PRICE = 'close'
COMMISSION = 0 # In percent

Don't change the below cell; Just run it.

This updates data of all the files in the folder. If you add a file, just rerun it again so that the database is updated.

In [3]:
connection_string = 'sqlite:///' + DBNAME
engine = create_engine(connection_string)
dl = DataLoader(DIRECTORY, mode='SQL', engine=engine, tablename=TABLENAME)
dl.load_data(parse_dates=['TIMESTAMP'], usecols=range(13)) # This actually loads data

In [4]:
# Now read the entire table back
df = pd.read_sql_table(TABLENAME, engine)

# Check the number of rows; would increase as you put more files in your folder
len(df)

41881

Don't change any cells below.
Just change the **PARAMETERS** above

Of course, you could always *hack the below code*

Backtesting
------------
1. Filter data only for our stock universe
2. Calculate daily returns for each of the stock
3. Carry the returns figure to the next day so that we can view them the next day
4. Group by each day, and apply our strategy
5. Append all the data
6. Calculate prices and performance
7. Evaluate metrics

In [5]:
symbols = pd.read_excel('universe.xlsx', sheet_name=UNIVERSE, header=None).values.ravel()

df = df[df['symbol'].isin(symbols)]
df = df[df['series'] == "EQ"].reset_index(drop=True)
df['ret'] = (df['close']/df['prevclose']) - 1
df = df.sort_values(by='timestamp')
df['ret'] = df.groupby('symbol')['ret'].transform(lambda x: x.shift(1))
grouped = df.groupby('timestamp')
collect = []
for name, group in grouped:
    temp = group.sort_values(by='ret', ascending=False).iloc[:5]
    collect.append(temp)
orders = pd.concat(collect)
    

In [6]:
def isPrice(price, high, low):
    if price >= low and price <=high:
        return True
    else:
        False

In [7]:
trading_capital = CAPITAL * LEVERAGE
orders['price'] = orders['open']
orders['stop_loss'] = (orders['price'] * (1+STOP_LOSS*0.01)).round(2)
orders['qty'] = (trading_capital/NUM_STOCKS/orders['price']).round()
orders['sell'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['price'], orders['high'], orders['low'], orders[REF_PRICE])]
orders['buy'] = [price if isPrice(price, high, low) else close for
                 price, high, low, close in 
                 zip(orders['stop_loss'], orders['high'], orders['low'], orders[REF_PRICE])]
orders['profit_per_unit'] = orders['sell'] - orders['buy']
orders['total_profit'] = orders['profit_per_unit'] * orders['qty']
cols = ['timestamp', 'symbol', 'buy', 'sell', 'profit_per_unit', 'total_profit']


## Metrics

In [8]:
orders.tail()[['symbol', 'open', 'high', 'low', 'close', 
               'qty', 'buy', 'sell', 'total_profit']]

,symbol,open,high,low,close,qty,buy,sell,total_profit
1060,DRREDDY,2510.0,2632.0,2495.00,2593.40,2.0,2610.40,2510.0,-200.80
1092,TECHM,768.0,776.5,741.50,745.65,5.0,745.65,768.0,111.75
1080,LUPIN,936.5,957.9,931.55,936.10,4.0,936.10,936.5,1.60
1089,TATAMOTORS,271.4,273.0,265.25,266.60,15.0,266.60,271.4,72.00
1064,HCLTECH,1053.0,1061.0,1044.60,1049.15,4.0,1049.15,1053.0,15.40


In [9]:
total_profit = orders['total_profit'].sum()
'Returns for the period = {:.2f}%'.format((total_profit/CAPITAL)*100)

'Returns for the period = 4.11%'

In [10]:
orders['total_profit'].describe()

count    110.000000
mean       7.468818
std       72.278348
min     -200.800000
25%      -35.837500
50%       12.450000
75%       46.237500
max      207.000000
Name: total_profit, dtype: float64

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
by_day = orders.groupby('timestamp').total_profit.sum()
(by_day.cumsum() + CAPITAL).plot(title = 'Portfolio by day')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.show()

<Figure size 640x480 with 1 Axes>

In [12]:
orders.set_index('timestamp').loc['2018-08-30']

,symbol,series,open,high,low,close,last,prevclose,tottrdqty,tottrdval,totaltrades,isin,ret,price,stop_loss,qty,sell,buy,profit_per_unit,total_profit
timestamp,,,,,,,,,,,,,,,,,,,,
2018-08-30,UPL,EQ,686.0,699.0,683.00,696.15,694.05,680.0,5979259,4.149426e+09,81679,INE628A01036,0.039517,686.0,713.44,6.0,686.0,696.15,-10.15,-60.90
2018-08-30,BAJFINANCE,EQ,2993.9,2993.9,2920.95,2926.75,2928.00,2985.5,1452996,4.284215e+09,88899,INE296A01024,0.018039,2993.9,3113.66,1.0,2993.9,2926.75,67.15,67.15
2018-08-30,SBIN,EQ,309.3,310.5,306.00,308.60,308.90,309.7,19935935,6.139214e+09,112027,INE062A01020,0.014246,309.3,321.67,13.0,309.3,308.60,0.70,9.10
2018-08-30,ONGC,EQ,176.9,181.9,176.15,177.45,178.20,177.0,11296260,2.023039e+09,84631,INE213A01029,0.013746,176.9,183.98,23.0,176.9,177.45,-0.55,-12.65
2018-08-30,BPCL,EQ,359.9,364.5,357.85,363.35,362.85,361.7,4065049,1.470797e+09,40564,INE029A01011,0.008785,359.9,374.30,11.0,359.9,363.35,-3.45,-37.95
